In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



df = pd.read_csv("aluminium_features_final.csv", parse_dates=["date"])

df = df.sort_values("date").reset_index(drop=True)

print("Dimensões:", df.shape)
print(df.dtypes)
display(df.head())
print(df.isna().sum())


Dimensões: (1304, 26)
date                datetime64[ns]
aluminum                   float64
copper                     float64
zinc                       float64
tin                        float64
lead                       float64
steel                      float64
oil                        float64
gas                        float64
dxy                        float64
usdcny                     float64
eurusd                     float64
brlusd                     float64
msci_em                    float64
xlb                        float64
pick                       float64
fxi                        float64
alcoa                      float64
century_aluminum           float64
hindalco                   float64
alum_pct                   float64
aluminum_usdton            float64
copper_usdton              float64
zinc_usdton                float64
tin_usdton                 float64
lead_usdton                float64
dtype: object


,date,aluminum,copper,zinc,tin,lead,steel,oil,gas,dxy,...,fxi,alcoa,century_aluminum,hindalco,alum_pct,aluminum_usdton,copper_usdton,zinc_usdton,tin_usdton,lead_usdton
0,2020-08-24,14.481213,1741.00,5.6186,45.130001,9.24,93.300003,39.994389,1.179802,39.667831,...,57.030403,72.065216,139.515625,3431.280029,-0.004630,1448.121262,38382.43420,561.859989,4513.000107,923.999977
1,2020-08-25,14.414170,1745.50,5.6106,45.860001,9.32,93.019997,40.396393,1.179454,39.827858,...,56.804012,71.972603,139.312500,3443.620117,-0.004630,1441.417027,38481.64210,561.059999,4586.000061,931.999969
2,2020-08-26,14.433324,1744.75,5.5074,45.639999,9.32,93.010002,40.503590,1.183502,39.756741,...,57.401665,72.009636,139.312500,3478.729980,0.001329,1443.332386,38465.10745,550.740004,4563.999939,931.999969
3,2020-08-27,14.213040,1746.25,5.6072,45.090000,9.29,93.010002,40.164124,1.184273,39.676723,...,57.247711,72.268883,138.968750,3484.550049,-0.015262,1421.304035,38498.17675,560.720015,4509.000015,928.999996
4,2020-08-28,14.481213,1777.25,5.5687,45.049999,9.90,92.370003,40.691193,1.181963,39.996769,...,57.926876,72.954094,139.250000,3508.010010,0.018868,1448.121262,39181.60895,556.869984,4504.999924,989.999962


date                0
aluminum            0
copper              0
zinc                0
tin                 0
lead                0
steel               0
oil                 0
gas                 0
dxy                 0
usdcny              0
eurusd              0
brlusd              0
msci_em             0
xlb                 0
pick                0
fxi                 0
alcoa               0
century_aluminum    0
hindalco            0
alum_pct            0
aluminum_usdton     0
copper_usdton       0
zinc_usdton         0
tin_usdton          0
lead_usdton         0
dtype: int64


Função: adddynamictargets
Adiciona targets dinâmicos (variáveis de previsão) para tarefas de regressão e classificação em séries temporais, com horizonte de previsão variável.

Objetivo
Criar, a partir de uma coluna de preço (“price_column”), três novos tipos de coluna para cada horizonte desejado:

Preço futuro (future_Xd): o valor da série X dias à frente daquela data.

Retorno percentual futuro (ret_Xd): o percentual de ganho/perda ao comprar agora e vender após X dias.

Classe (“bin”) (bin_Xd): uma classificação do retorno em faixas com significado de mercado (ex: “Sobe muito”, “Na mesma”, “Cai muito”), útil para tarefas de classificação.

Parâmetros
df (pd.DataFrame): DataFrame original, deve conter pelo menos uma coluna de preço.
price_column (str): nome da coluna cujo preço será usado para calcular os targets (padrão: "aluminum_usdton").
horizon (int): quantos dias à frente olhar para construir o target (ex: 1, 5, 30, 90).
prefix (str): texto colocado antes do nome dos novos targets (opcional, útil para múltiplos ativos).
Novas Colunas Criadas
Para cada valor de horizon:

future_Xd: valor da coluna price_column X dias à frente.
ret_Xd: retorno percentual de X dias:
(preço_future_Xd - preço_atual) / preço_atualfutureXd - preçoatual) / preçoat

bin_Xd: classificação do retorno futuro (%), seguindo as faixas:
“Sobe muito”: retorno > +10%
“Sobe”: +3% < retorno ≤ +10%
“Na mesma”: -3% ≤ retorno ≤ +3%
“Cai”: -10% < retorno < -3%
“Cai muito”: retorno ≤ -10%

In [7]:
def add_dynamic_targets(df, price_column="aluminum_usdton", horizon=1, prefix=""):
    """
    Adiciona targets de previsão numérica e classificação por bins, com horizonte variável.
    - df: dataframe original.
    - price_column: string do nome do preço alvo.
    - horizon: int, janela em dias para olhar o futuro.
    - prefix: string prefixo nas novas colunas.
    """
    # Shift negativo para encontrar o preço futuro em 'horizon' dias
    df[f"{prefix}future_{horizon}d"] = df[price_column].shift(-horizon)
    
    # Retorno percentual (%)
    df[f"{prefix}ret_{horizon}d"] = (df[f"{prefix}future_{horizon}d"] - df[price_column]) / df[price_column]
    
    # Função para classificar bins
    def classify_bin(x):
        if x > 0.10:
            return "Sobe muito"
        elif x > 0.03:
            return "Sobe"
        elif x >= -0.03:
            return "Na mesma"
        elif x > -0.10:
            return "Cai"
        else:
            return "Cai muito"
    
    df[f"{prefix}bin_{horizon}d"] = df[f"{prefix}ret_{horizon}d"].apply(classify_bin)
    
    return df

# Exemplo de uso para 1 dia, 5 dias e 30 dias  
horizons = [1, 5, 30, 90]
for h in horizons:
    df = add_dynamic_targets(df, price_column="aluminum_usdton", horizon=h)

# Checagem
df[[f"future_{horizons[0]}d", f"ret_{horizons[0]}d", f"bin_{horizons[0]}d", f"future_{horizons[1]}d", f"bin_{horizons[2]}d"]].head(10)


,future_1d,ret_1d,bin_1d,future_5d,bin_30d
0,1441.417027,-0.004630,Na mesma,1400.233364,Cai muito
1,1443.332386,0.001329,Na mesma,1445.248032,Cai muito
2,1421.304035,-0.015262,Na mesma,1405.022430,Cai muito
3,1448.121262,0.018868,Na mesma,1362.881088,Cai muito
4,1400.233364,-0.033069,Cai,1379.162979,Cai muito
5,1445.248032,0.032148,Sobe,1371.022034,Cai muito
6,1405.022430,-0.027833,Na mesma,1362.881088,Cai muito
7,1362.881088,-0.029993,Na mesma,1350.430584,Cai muito
8,1379.162979,0.011947,Na mesma,1319.782543,Cai muito
9,1371.022034,-0.005903,Na mesma,1338.937569,Cai muito


In [ ]:
import numpy as np
import pandas as pd

ativos = [
    "aluminum", "copper", "zinc", "tin", "lead", "steel", "oil", "gas", "dxy", 
    "usdcny", "eurusd", "brlusd", "msci_em", "xlb", "pick", "fxi", 
    "alcoa", "century_aluminum", "hindalco", "aluminum_usdton",
    "copper_usdton", "zinc_usdton", "tin_usdton", "lead_usdton"
]
# Escolha de janelas típicas para rolling: 5, 10, 21, etc (dias úteis ~ semanas/meses)
rolling_windows = [5, 10, 21] 
lags = [1, 3, 5, 10, 21]      # Idem para defasagens

# 1️Retornos diários 
retornos = {}
logretornos = {}
for ativo in ativos:
    retornos[f"{ativo}_ret1"] = df[ativo].pct_change()
    logretornos[f"{ativo}_logret1"] = np.log(df[ativo] / df[ativo].shift(1))

# 2️Rolling features (médias, volatilidades, máximos, mínimos)
ma = {}
std = {}
maximos = {}
minimos = {}
for ativo in ativos:
    for win in rolling_windows:
        ma[f"{ativo}_ma{win}"] = df[ativo].rolling(window=win).mean()
        std[f"{ativo}_std{win}"] = df[ativo].rolling(window=win).std()
        maximos[f"{ativo}_max{win}"] = df[ativo].rolling(window=win).max()
        minimos[f"{ativo}_min{win}"] = df[ativo].rolling(window=win).min()

# Lags temporais
lags_dict = {}
for ativo in ativos:
    for lag in lags:
        lags_dict[f"{ativo}_lag{lag}"] = df[ativo].shift(lag)

# Z-score e indicador acima/dentro MM para alumínio
alum_zscores = {}
alum_above = {}
for win in rolling_windows:
    ma_col = f"aluminum_usdton_ma{win}"
    std_col = f"aluminum_usdton_std{win}"
    alum_zscores[f"alum_zscore_ma{win}"] = (df["aluminum_usdton"] - ma[ma_col]) / std[std_col]
    alum_above[f"alum_above_ma{win}"] = (df["aluminum_usdton"] > ma[ma_col]).astype(int)

# 5️⃣ JUNTANDO TUDO
df = pd.concat([
    df,
    pd.DataFrame(retornos, index=df.index),
    pd.DataFrame(logretornos, index=df.index),
    pd.DataFrame(ma, index=df.index),
    pd.DataFrame(std, index=df.index),
    pd.DataFrame(maximos, index=df.index),
    pd.DataFrame(minimos, index=df.index),
    pd.DataFrame(lags_dict, index=df.index),
    pd.DataFrame(alum_zscores, index=df.index),
    pd.DataFrame(alum_above, index=df.index),
], axis=1)

# Pandas: defragmentar por segurança/memória
df = df.copy()

# VALIDAÇÃO DE FEATURES CRIADAS
print("Colunas criadas para 'aluminum_usdton':")
for part in ['ret1', 'logret1', 'ma5', 'std5', 'max10', 'min21', 'lag3', 'lag10', 'zscore_ma21', 'above_ma10']:
    colname = [c for c in df.columns if f"aluminum_usdton_{part}" in c or f"alum_{part}" in c]
    for c in colname:
        print(f" - {c} | Nulos: {df[c].isna().sum()} | min: {df[c].min()}, max: {df[c].max()}")

print("\nTotal de colunas finais:", len(df.columns))


Colunas criadas para 'aluminum_usdton':
 - aluminum_usdton_ret1 | Nulos: aluminum_usdton_ret1    1
aluminum_usdton_ret1    1
dtype: int64 | min: aluminum_usdton_ret1   -0.169447
aluminum_usdton_ret1   -0.169447
dtype: float64, max: aluminum_usdton_ret1    0.152263
aluminum_usdton_ret1    0.152263
dtype: float64
 - alum_ret1_minus_copper_ret1 | Nulos: 1 | min: -0.177632222963502, max: 0.1569521532711744
 - aluminum_usdton_ret1 | Nulos: aluminum_usdton_ret1    1
aluminum_usdton_ret1    1
dtype: int64 | min: aluminum_usdton_ret1   -0.169447
aluminum_usdton_ret1   -0.169447
dtype: float64, max: aluminum_usdton_ret1    0.152263
aluminum_usdton_ret1    0.152263
dtype: float64
 - aluminum_usdton_logret1 | Nulos: aluminum_usdton_logret1    1
aluminum_usdton_logret1    1
dtype: int64 | min: aluminum_usdton_logret1   -0.185663
aluminum_usdton_logret1   -0.185663
dtype: float64, max: aluminum_usdton_logret1    0.141728
aluminum_usdton_logret1    0.141728
dtype: float64
 - aluminum_usdton_logret1 

In [ ]:
# Spreads principais de mercado: diferenças absolutas
df['copper_minus_aluminum'] = df['copper_usdton'] - df['aluminum_usdton']
df['oil_minus_gas'] = df['oil'] - df['gas']
df['tin_minus_zinc'] = df['tin_usdton'] - df['zinc_usdton']
df['aluminum_minus_lead'] = df['aluminum_usdton'] - df['lead_usdton']
df['alum_pct_minus_oil'] = df['alum_pct'] - df['oil']

# Spreads ajudam o modelo a captar movimentos relativos/setoriais e possíveis arbitragens naturais do mercado.


In [ ]:
# Ratios entre ativos principais
df['copper_div_aluminum'] = df['copper_usdton'] / df['aluminum_usdton']
df['oil_div_gas'] = df['oil'] / df['gas']
df['tin_div_copper'] = df['tin_usdton'] / df['copper_usdton']
df['lead_div_aluminum'] = df['lead_usdton'] / df['aluminum_usdton']

# Variações nesses "ratios" costumam antecipar movimentos de reversão ou tendência relativa entre commodities.


In [33]:
print(df.columns.duplicated().any())  # Deve ser False!


True


In [34]:
df = df.loc[:, ~df.columns.duplicated()]
print(df.columns.duplicated().any())  # Deve agora ser False!

False


In [ ]:
df['alum_ret1_minus_copper_ret1'] = df['aluminum_usdton_ret1'] - df['copper_usdton_ret1']
df['zinc_ret1_minus_lead_ret1'] = df['zinc_usdton_ret1'] - df['lead_usdton_ret1']
df['oil_ret1_minus_steel_ret1'] = df['oil_ret1'] - df['steel_ret1']
df['pick_ret1_minus_msci_em_ret1'] = df['pick_ret1'] - df['msci_em_ret1']


In [ ]:
# Médias móveis para alumínio (ajuste para outros ativos replicando o bloco!)
df['alum_ma5'] = df['aluminum_usdton'].rolling(window=5).mean()
df['alum_ma21'] = df['aluminum_usdton'].rolling(window=21).mean()
df['alum_cross_ma5_ma21'] = (df['alum_ma5'] > df['alum_ma21']).astype(int)

# Detecta tendências de curto vs. longo prazo — feature clássica em momentum.


In [ ]:
# Contagem de quanto tempo está acima da média de 10 períodos
# (Pressupõe que "alum_above_ma10" já existe no DF!)
df['alum_above_ma10_roll10'] = df['alum_above_ma10'].rolling(window=10).sum()

# Mede persistência ("momentum") — quanto mais tempo acima da média, maior a chance de continuidade ou reversão.


In [ ]:
# Overbought / Oversold usando z-score de 21 períodos
# (Pressupõe que 'alum_zscore_ma21' já existe!)
df['alum_oversold'] = (df['alum_zscore_ma21'] < -1.5).astype(int)
df['alum_overbought'] = (df['alum_zscore_ma21'] > 1.5).astype(int)
s
# Permite ao modelo identificar situações estatisticamente extremas, sugerindo reversão.


In [ ]:
# Data deve estar em formato datetime64[ns]:
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['weekday'] = df['date'].dt.weekday

# Fim de mês
df['end_of_month'] = (df['date'] == (df['date'] + pd.tseries.offsets.MonthEnd(0))).astype(int)

# Sazonalidade financeira é real: ciclos de produção, rebalanceamento de fundos, virada de contratos, etc, afetam commodities.


In [39]:

df.to_csv('aluminium_full_featured.csv', index=False)


Documentação das Variáveis do Dataset de Commodities ALUMÍNIO

### 1. Variáveis de Tempo e Calendário
date: Data da observação (formato datetime).
month: Número do mês da observação (1–12).
quarter: Número do trimestre (1–4).
weekday: Dia da semana (0 = Segunda… 6 = Domingo).
endofmonth: Indicador binário do último dia útil do mês (1 = sim, 0 = não).



### 2. Preços à Vista de Commodities e Índices
aluminum, copper, zinc, tin, lead: Preços spot das respectivas commodities (unidade padrão internacional, normalmente USD/ton).
steel, oil, gas: Preços spot de aço, petróleo (oil), gás natural (gas).
dxy, usdcny, eurusd, brlusd: Índices e pares cambiais: Dólar index, USD/CNY, EUR/USD, BRL/USD.
msci_em: Índice MSCI Emerging Markets.
xlb, pick, fxi: ETFs setoriais/setores commodities (ex: XLB = Materials, PICK = Mineração/metais).
alcoa, century_aluminum, hindalco: Preço das ações de empresas do setor de alumínio.
alum_pct: Participação/proxy de alumínio em índice ou carteira (se aplicável).


### 3. Preços em USD/ton (Commodities)
aluminumusdton, copperusdton, zincusdton, tinusdton, lead_usdton: Preço futuro/spot padronizado em USD por tonelada.

### 4. Targets, Retornos e Labels (Janelas Diferentes)
future1d, ret1d, bin_1d: Preço futuro (em 1 dia), retorno (log ou pct) e classificador (label categórico) para curto prazo.
future5d, ret5d, bin_5d: … para 5 dias.
future30d, ret30d, bin_30d: … para 30 dias.
future90d, ret90d, bin_90d: … para 90 dias.


### 5. Retornos e Logretornos de 1 Período
[ativo]_ret1: Retorno percentual de 1 período da commodity/ativo.
[ativo]_logret1: Retorno logarítmico de 1 período da commodity/ativo.

### 6. Rolling Windows (Features Técnicas de Curto, Médio e Longo Prazo)
Para cada ativo são criadas as seguintes, em “janelas” de 5, 10 e 21 períodos:

[ativo]_maX: Média móvel de X períodos.
[ativo]_stdX: Desvio padrão (volatilidade) dos retornos/preços para X períodos.
[ativo]_maxX/minX: Máximo e mínimo nos últimos X períodos.
Replicado para: commodities, moedas, ações, ETFs.

### 7. Lags
[ativo]_lagX: Preço fechado ou valor do ativo X dias atrás.
Cobre lags de 1, 3, 5, 10 e 21 períodos para quase todos os ativos principais.

### 8. Features Avançadas de Momentum, Reversão e Persistência
alumzscoremaX: Z-score do preço de alumínio em relação à média de X períodos e seu desvio padrão.
alumabovemaX: Binário, 1 se preço do alumínio acima da média X, 0 caso contrário.
alumabovema10_roll10: Soma dos últimos 10 dias em que preço ficou acima da média móvel de 10 períodos (persistência).
alumoversold, alumoverbought: Sinal binário de sobrevendido/sobrecomprado via z-score de 21 períodos (<-1.5 ou >1.5).
alumma5, alumma21: Médias móveis específicas para alumínio.
alumcrossma5_ma21: Binário: 1 se ma5 > ma21, 0 caso contrário (cruzamento de médias, sinal clássico de momentum).

### 9. Spreads, Relações e Proporções Entre Ativos
copperminusaluminum, zincminusaluminum, oilminusgas, etc: Diferenças absolutas de preço entre pares de ativos.
copperdivaluminum, zincdivaluminum, oildivgas, tindivcopper, leaddivaluminum, etc: Razões (proporções) entre preços de ativos.
tinminuszinc, aluminumminuslead, alumpctminus_oil, etc: Outras variações de spreads e proporções.
alumret1minuscopperret1, zincret1minusleadret1, oilret1minussteelret1, pickret1minusmsciem_ret1: Diferença de retornos entre pares de ativos, normalizando movimentações setoriais.

### 10. Outras Features Técnicas
[ativo]_maX, stdX, maxX, minX: Repetido para principais ativos do universo.
[ativo]_lagX: Lags de preços ou retornos para diferentes prazos, para entrada direta em modelos de machine learning.
aluminumusdtonlagX, copperusdtonlagX, etc: Mesma lógica mas usando preço “em dólar por tonelada”.

### 11. Observação Sobre Codificação e Sazonalidade
Para variáveis sazonais como month, quarter, weekday, endofmonth, importante considerar o uso de encoding cíclico (seno/cosseno) para evitar que modelos “vejam” Dezembro como distante de Janeiro.
Feriados ou eventos macro relevantes podem e devem ser adicionados (ex: “isholiday”, “rollcontract”, etc).

### 12. Exemplo do Núcleo para Alumínio
aluminum, aluminumusdton, aluminumusdtonret1, aluminumusdtonlogret1, aluminumusdton_ma5, etc…
alumzscoremaX, alumabovemaX, alumabovema10roll10, alumoversold, alumoverbought, alumma5, alumma21, alumcrossma5ma21
aluminumlag1, aluminumlag3, …, aluminumusdtonlagX


### 13. Observações Finais
Dataset cobre o essencial de macro, commodities, FX, índices, ETFs, empresas setoriais, features técnicas, spreads, rolling windows, lags, variáveis sazonais e sinais clássicos de trading quant.
Cada camada pode ser customizada, expandida ou agregada via novas features a depender do objetivo (modelos supervisionados, clusterização, visualização, monitoramento de regime, etc).